### Install ini terlebih dahulu

In [ ]:
!pip install tradingWithPython

In [ ]:
%pylab inline

In [ ]:
import requests # interaction with the web
import os  #  file system operations
import yaml # human-friendly data format
import re  # regular expressions
import pandas as pd # pandas... the best time series library out there
import datetime as dt # date and time functions
import io 
import time

## Ambil Cookie yahoo dan token

Cookie ini diperlukan untuk download dan bisa digunakan setahun


In [ ]:
# search with regular expressions

# "CrumbStore":\{"crumb":"(?<crumb>[^"]+)"\}

url = 'https://uk.finance.yahoo.com/quote/AAPL/history' # url for a ticker symbol, with a download link
r = requests.get(url)  # download page

txt = r.text # extract html


cookie = r.cookies['B'] # the cooke we're looking for is named 'B'
print('Cookie: ', cookie)

# Now we need to extract the token from html. 
# the string we need looks like this: "CrumbStore":{"crumb":"lQHxbbYOBCq"}
# regular expressions will do the trick!

pattern = re.compile('.*"CrumbStore":\{"crumb":"(?P<crumb>[^"]+)"\}')

for line in txt.splitlines():
    m = pattern.match(line)
    if m is not None:
        crumb = m.groupdict()['crumb']
        
        
print('Crumb=',crumb)

In [ ]:
# create data directory in the user folder
dataDir = os.path.expanduser('~')+'/twpData'

if not os.path.exists(dataDir):
    os.mkdir(dataDir)


# save data to YAML file
data = {'cookie':cookie,'crumb':crumb}

dataFile = os.path.join(dataDir,'yahoo_cookie.yml')

with open(dataFile,'w') as fid:
    yaml.dump(data,fid)

## Masukin tanggal mulai dan tanggal akhir

menggunakan timestamp()

`https://query1.finance.yahoo.com/v7/finance/download/SPY?period1=1463754366&period2=1495290366&interval=1d&events=history&crumb=DB/mJy8XKWr`

In [ ]:
# start with tuples ...
sDate = (2001,1,1)
eDate = (2010,4,21)
sq = 'WOMF.JK'
 
dt.datetime(*sDate).timestamp() # convert to seconds since epoch


## Ambil data

Percobaan ambil data

In [ ]:
# prepare input data as a tuple
data = (int(dt.datetime(*sDate).timestamp()),
        int(dt.datetime.now().timestamp()), 
        crumb)


url = "https://query1.finance.yahoo.com/v7/finance/download/{3}?period1={0}&period2={1}&interval=1d&events=history&crumb={2}".format(*data,sq)


print(url)
data = requests.get(url, cookies={'B':cookie})

In [ ]:
buf = io.StringIO(data.text) # create a buffer
df = pd.read_csv(buf,index_col=0) # convert to pandas DataFrame
df.head()

## Buat fungsi untuk men-mendownload lebih banyak

In [ ]:
def download_stock(stock):
    # start with tuples ...
    sDate = (2001,1,1)
    eDate = (2018,12,21)
    sq = stock
    path='./stock/'
 
    dt.datetime(*sDate).timestamp() # convert to seconds since epoch
    

    # prepare input data as a tuple
    data = (int(dt.datetime(*sDate).timestamp()),
            int(dt.datetime(*eDate).timestamp()), 
            crumb)


    url = "https://query1.finance.yahoo.com/v7/finance/download/{3}?period1={0}&period2={1}&interval=1d&events=history&crumb={2}".format(*data,sq)

    try:
        data = requests.get(url, cookies={'B':cookie})
        buf = io.StringIO(data.text) # create a buffer
        df = pd.read_csv(buf,index_col=0) # convert to pandas DataFrame
        df.to_csv(path+sq+".csv")
        print(sq," downloaded")
    except:
        print("Error reading/writing: ",sq)
        
    time.sleep(2)


In [ ]:
# kompilasi list
lst = pd.read_csv('all_list.csv')
slist = list(lst['Kode']+".JK")

In [ ]:
## list yang akan didownload
slist

# Download all stocks

In [ ]:
## MENDOWNLOAD SEMUA YANG ADA DI SLIST
%%time
for sl in slist:
    download_stock(sl)

## CHECK PEMENANG TERAKHIR - TOP GAIN -  DARI SAHAM

In [ ]:
import glob
sl = glob.glob('./stock/*.csv')

In [ ]:
sql = []
for i in sl:
    df = pd.read_csv(i)
    
    try:
        dlast = df.iloc[-1]['Close']
        dprev = df.iloc[-2]['Close']

        if (dlast>dprev*1.2):
            print(i)
            sql.append(i)
    except:
        pass

In [ ]:
for i in sql:
    stk = pd.read_csv(i)
    stk['Close'].plot()

In [ ]:
stk['Close'].plot()

In [ ]:
stk.iloc[-100:-1]['Close'].plot()